In [1]:
from scripts.functions import *
from parameters import *

<IPython.core.display.Javascript object>

to use your own api keys and parameters, copy paste the `parameters.py.dist` file in the same folder and remove the `.dist` extention. You can then replace the string with your own keys. only the .dist will be pushed to the dist git rep. 

# 1. Search items


### Create bounding box from centroids

In [ ]:
# Specify a projected EPSG for the centroids file
# EPSG:21148 is for Indonesia.
# samples_gdf = read_from_centroids(projected_epsg='EPSG:21148', buffer=350, sep=';')

In [3]:
samples_gdf = gpd.read_file('shp/IndonesiaNFI_14ha.shp')
samples_gdf = samples_gdf.set_index(['OBJECTID'])
samples_gdf = samples_gdf[['geometry']]
samples_gdf.index.names = ['id']

ERROR:fiona._env:Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.


In [15]:
samples_gdf[:5]

/usr/local/lib/python3.6/dist-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,geometry
id,


In [4]:
len(samples_gdf)

3928

In [ ]:
# samples_gdf = gpd.read_file('shp/Indonesia_hexa_505_rand_21ha.shp')
# samples_gdf = samples_gdf.set_index(['global_id'])
# samples_gdf = samples_gdf[['geometry']]
# samples_gdf.index.names = ['id']

### Connect to client

In [5]:
client = api.ClientV1(api_key=PLANET_API_KEY)

### Define filters

In [ ]:
# define test data for the filter
start_date = datetime.datetime(2017, 1, 1)
stop_date = datetime.datetime(2020, 12, 31)
cloud_cover_lte = 0.10
minimum_covered_area = 90 # included

### Define scores

In [ ]:
# item_type_score
item_type_score = {
    'PSScene4Band':10, 
    'PSScene3Band':0, 
    'PSOrthoTile':8,
    'REOrthoTile':0,
    'SkySatScene':0,
}

# season score
months_score = {
    1: 0, 7:8,
    2: 0, 8:10,
    3: 0, 9:10,
    4: 7, 10:8,
    5: 7, 11:0,
    6: 7, 12:0,
}

# cloud_score

def cloud_score(cloud_cover):
    """ Define the cloud cover threshold and score
    
    1 = 1%
    
    """
    cloud_cover = cloud_cover*100
    
    if cloud_cover == 0:
        return 10
    elif cloud_cover <= 5 and cloud_cover > 0:
        return 5
    else:
        return 0
    
# Covered area

def cover_score(covered_area):
    """Define the cover area threshold and score
    """
    covered_area = covered_area*100
    
    if covered_area >= 99:
        return 10
    
    elif covered_area >= 95:
        return 5
    
    else:
        return 0
    


## OPTION 1.2 Get items for all plots and store into a big df


### Selection method
The loop will search all the images between the given start-end date, and the minimum cloud coverage.<br>
After that it will calculate the sample covered area with the image item footprint and then will remove items which are under the given threshold.<br>
The next step is rank the items by the selected parameters <br>
#### Temporal selection
The user has to select the desired time span for get the images: 1 per year, 1 per month, or one every x images.

In [ ]:
# If by_month is True, one image per month will be chosen, otherwise one per year.
# By default it will process only one image per year

by_month = False
by_every = 1

### Loop over all plots in parellel
Loop over all plots and get the items.

In [ ]:
def run_multiprocess(index, row, srch_log_file, by_month=False, by_every=0, skip_items=None):
    
    aoi_geometry = json.loads(dumps(row.geometry))
    sample_id = row.name
    
    if by_every:
        pickle_df_name = os.path.join(OUT_PIKL_PATH, str(sample_id)+'_every.p')
    elif by_month:
        pickle_df_name = os.path.join(OUT_PIKL_PATH, str(sample_id)+'_month.p')
    else:
        pickle_df_name = os.path.join(OUT_PIKL_PATH, str(sample_id)+'_year.p')
        
    if not os.path.exists(pickle_df_name):
        request = build_request(aoi_geometry, start_date, stop_date, cloud_cover_lte)

        try:
            print(f'Starting {sample_id}')
            items = get_items(sample_id, request, client)
            # Transform items into a pandas dataframe with useful columns
            metadata_df = get_dataframe(items)
            
            # Skip items with errors
            if skip_items:
                metadata_df = metadata_df[~metadata_df.id.isin(skip_items)]
            
            # Mutate metadata_df and add the percentage of cover area
            add_cover_area(metadata_df, samples_gdf)

            # Remove items that are under the minimum_covered_area threshold
            metadata_df = metadata_df[metadata_df.cover_perc >= (minimum_covered_area/100)]

            # Create a score for each item
            scored_items = score_items(metadata_df, item_type_score, months_score, cloud_score, cover_score)
            
            if by_every:
                # Filter scored_items and get one item every x items
                selected_items = get_one_item_every_x(scored_items, every=by_every)
            
            elif by_month:
                # Filter scored_items and get only one per month
                selected_items = get_one_item_per_month(scored_items)
            else:
                # Filter scored_items and get only one per year
                selected_items = get_one_item_per_year(scored_items)
            
            # Save into a pickled file
            selected_items.to_pickle(pickle_df_name)
            
            print(f'{sample_id} pickled.')
            
        except Exception as e:
            print(f'there was an error with the sample {sample_id}, please check the log files.')
            with open(srch_log_file, 'a') as lf:
                lf.write(f'"{sample_id}":{e}\n')

    else:
        print(f'Search for {sample_id} already saved.')

In [ ]:
len(samples_gdf)

### Skip error items from logs
Uncomment the next cell if you have a log file with "no access to assets" elements, so the process will skip them.

<br> If you are using this option, please delete the previous searches pickled files from the failed samples (search failed samples with the commands in step 4) 

In [ ]:
skip_items = None
# skip_items = get_no_access_assets_from_log('logs/order_logs_20200916_15_02.txt')
# len(skip_items)

In [ ]:
%%time
if __name__ == '__main__':

    # Create a log file
    now = datetime.datetime.now()
    formated_now = now.strftime('%Y%m%d_%H_%M')
    srch_log_file = os.path.join(LOG_PATH, f'search_logs_{formated_now}.txt')
    
    # Set the number of parallel processes
    pool = multiprocessing.Pool(15)
    
    for index, row in samples_gdf.iterrows():
        pool.apply_async(run_multiprocess, args=(index, row, srch_log_file, by_month, by_every, skip_items))
        
    pool.close()
    pool.join()

### Read all the pickled files, merge and store them in a big df

In [ ]:
pickled_files = glob.glob(os.path.join(OUT_PIKL_PATH,'every_500_hexa/*every.p'))
len(pickled_files)

In [ ]:
# all_df.to_pickle('searches/merged/ALL_Rand500_hexa.p')

In [ ]:
# df_500_all = pd.read_pickle('searches/merged/ALL_Rand500_hexa.p')
# df_500_all = pd.read_pickle('searches/merged/ALL_Rand500_PSS4.p')
# df_nfi = pd.read_pickle('searches/merged/NFI_PSS4cclt001.p')
# df_500 = pd.read_pickle('searches/merged/Indonesia_hexa_500plots_PSS4cclt001.p')

In [6]:
all_df = pd.read_pickle('searches/merged/df_NFI_clear90_nulls.p')

In [7]:
len(all_df)

190454

# 2. Order assets
### Create json request

In [26]:
products_bundles = {

    # Is not possible to ask for analytic_dn in PSScene3Band, so the next option is visual
    # for more info go to https://developers.planet.com/docs/orders/product-bundles-reference/
    'PSScene3Band': "analytic,visual",
    'PSScene4Band': "analytic_udm2,analytic_sr,analytic",
    'PSOrthoTile': "analytic_5b_udm2,analytic_5b,analytic_udm2,analytic,visual",
    'REOrthoTile': "analytic,visual",
}

In [64]:
# To create the order we need a dataframe with filtered items,
# and a samples_gdf with sample_id and geometry to clip each item.

# Build an order for each sample and store in a orders_list
orders = []
samples_ids = list(all_df.sample_id.unique())
for idx, row in samples_gdf.iterrows():
    if idx in samples_ids:
        order = build_order_from_metadata(all_df, idx, row, products_bundles)
        orders.append(order)

In [66]:
len(orders)

3928

In [67]:
def multiprocess_order(new_order, ordered_sample_names, orders_info):
    # Make sure that the sample is not already downloaded
    sample_name = new_order['name']
    if sample_name not in ordered_sample_names:

        try:
            # The following line will create the order in the server
            @backoff.on_exception(backoff.expo,planet.api.exceptions.OverQuota, max_time=360)
            def place_order():
                return client.create_order(new_order).get()

            order_info = place_order()
            order_id = order_info['id']
            sample_name = order_info['name']

            print(f'order {order_id} with {sample_name} has been placed.')

        except Exception as e:
            with open(ordr_log_file, 'a') as lf:
                print(f'there was an error with the sample {sample_name}, please check the log files.')
                lf.write(f'Sample {sample_name}:{e}\n')


In [68]:
%%time
if __name__ == '__main__':

    # Create a log file
    now = datetime.datetime.now()
    formated_now = now.strftime('%Y%m%d_%H_%M')
    ordr_log_file = os.path.join(LOG_PATH, f'order_logs_{formated_now}.txt')
    
    # Request the existing orders and store their sample_id (name)
    current_server_orders = get_existing_orders(client)
    ordered_sample_names = [order['name'] for order in current_server_orders]
    
    # Set the number of parallel processes
    pool = multiprocessing.Pool(10)
    
   
    for new_order in orders:
        pool.apply_async(multiprocess_order, args=(new_order, ordered_sample_names, ordr_log_file))
        
    pool.close()
    pool.join()
print('Finished')

there was an error with the sample sample_3, please check the log files.there was an error with the sample sample_7, please check the log files.there was an error with the sample sample_6, please check the log files.there was an error with the sample sample_5, please check the log files.there was an error with the sample sample_2, please check the log files.
there was an error with the sample sample_4, please check the log files.there was an error with the sample sample_1, please check the log files.

there was an error with the sample sample_102, please check the log files.there was an error with the sample sample_101, please check the log files.there was an error with the sample sample_103, please check the log files.





order dbcc8cf9-3446-42ad-b495-02a4b8fd731b with sample_105 has been placed.
order e343a746-eace-4c60-9767-3296e89cebec with sample_104 has been placed.
order 441728a0-621a-4bbb-b013-925df4ef30a1 with sample_111 has been placed.
order a9b7dca6-b27d-4c4d-a606-3f8ca79

order e6b7a8c9-14aa-47e6-a6c7-13d0d1c427f3 with sample_204 has been placed.
order 3ed30689-3556-42b5-a8b3-28a5ee658c5a with sample_206 has been placed.
order 7a0c5dba-b1a3-4038-b189-0f4fb9f4db66 with sample_207 has been placed.
order 4f8ba9ce-506e-4793-8bd1-1697b9b45cb5 with sample_201 has been placed.
order 804bdb13-193c-4a12-90a3-92bb935cc266 with sample_209 has been placed.
order e066c45d-b907-497a-be18-99f23060483c with sample_205 has been placed.
order 824cda49-9529-4c2f-887c-89f8fe70b3ca with sample_203 has been placed.
order 82051852-537d-4464-aeff-34d5ce151edc with sample_208 has been placed.
order f308cfe7-4b27-425a-beda-0dfac4d6e6f5 with sample_210 has been placed.
order 4c81fc99-86f4-47f7-8941-5caf11eb8257 with sample_211 has been placed.
order dc4b7d6f-9a3a-476d-a7b8-1ca6dcd05ba4 with sample_212 has been placed.
order c675519e-da6a-4228-87c7-82a9c443f657 with sample_219 has been placed.
order a4b5f174-25fa-42be-afb3-cec62c3ba95e with sample_216 has been placed.
order 1ab9c0

order e8585a27-51e8-4b7c-9945-4e466f9fabcc with sample_314 has been placed.
order 640f0a7b-31e7-4107-9ece-d857385cc3f2 with sample_313 has been placed.
order 8098349f-6796-4316-a36b-3e172c260b0b with sample_315 has been placed.
order 3515476e-cd5d-4226-b1d4-dc7d7d32629e with sample_316 has been placed.
order 596ebf28-3ec2-48da-bdb1-3011735ccc6d with sample_308 has been placed.
order f122f8ae-46a2-40da-8065-f2e6dcb010a2 with sample_311 has been placed.
order f91d873e-0990-475f-9103-f12591e951cd with sample_317 has been placed.
order 7c9b0ad8-6314-439a-98bc-b318b64621d6 with sample_318 has been placed.
order df6c2f7f-d7bf-4c9b-9167-6e28868053cf with sample_319 has been placed.
order dff3e2d8-2567-4b02-86c4-b7d3484f0e87 with sample_310 has been placed.
order 94de9a53-5266-4f5b-b0b1-38bd3078b30c with sample_320 has been placed.
order d0dc9d39-222d-4b3a-9dbb-82ceda830589 with sample_323 has been placed.
order d66868e8-8f8c-4a3e-8c64-b4781513e1a4 with sample_322 has been placed.
order 8340d9

order f1c55b33-0a1d-4bef-94f5-0ba17be3e69e with sample_417 has been placed.
order 5193abef-064a-437f-9169-42a2b3e51346 with sample_418 has been placed.
order c184a4e7-42b0-43f9-840d-501d80dc0af8 with sample_419 has been placed.
order d11e50ef-88b8-46dd-99d8-fb2594ff6afb with sample_411 has been placed.
order e474f96c-8b15-4f2e-a823-771038670e9a with sample_420 has been placed.
order 6698509b-2d65-4c26-bed9-5a99e15ecabb with sample_428 has been placed.
order 097db135-4f31-4c35-bae7-be2803c838b0 with sample_429 has been placed.
order eec6b262-0338-4b02-b5f9-c6372626ef99 with sample_424 has been placed.
order b869800b-526d-4c65-8135-6f453060e4ff with sample_427 has been placed.
order e09efece-39a8-47f2-a2ac-9acc30150c02 with sample_426 has been placed.
order 1a9f8439-7dbb-4a21-9707-359efaece0e0 with sample_425 has been placed.
order 768ab8a2-d2d8-4193-93c2-8f53449794a0 with sample_421 has been placed.
order 078cfa16-96b5-4d6b-9a69-8fd695310fdd with sample_431 has been placed.
order bf5fee

order 77a64e22-5827-4031-865f-f34e4bed3503 with sample_520 has been placed.
order 8a14045a-fa2b-4e52-ae7c-1eadefdbbbce with sample_524 has been placed.
order c87d5878-f431-4ab5-86ea-d781b62dde1a with sample_529 has been placed.
order 04087604-21fe-4988-8bed-e78fb4052a2e with sample_533 has been placed.
order 2f186484-7ce8-4f32-8ed8-293e4b152e8d with sample_530 has been placed.
order 01e44852-0dc8-482d-bcbd-7196d802c524 with sample_528 has been placed.
order 98e6e757-d6d9-444a-bd76-f6a5254c5687 with sample_531 has been placed.
order 23fa7f09-c9c6-4343-b6e6-ec3889a75732 with sample_526 has been placed.
order d0e7fc5e-f13f-476c-99cc-8f24df4d12ee with sample_535 has been placed.
order 72b93fbb-85dd-4819-9f11-32fad9f1fa11 with sample_538 has been placed.
order 4079d733-14d1-4fba-a2a6-52c64c8ea35f with sample_525 has been placed.
order f2c5cfe7-f9d1-4b14-92da-e1db397d53b7 with sample_534 has been placed.
order 1ec5ff3b-55e3-45cb-b50d-61e118c0b3c9 with sample_537 has been placed.
order 62d074

...

order ed06de03-cdc4-41ba-9043-cb6425ea6cc1 with sample_3497 has been placed.
order 660cb0fb-ac05-4b42-b12d-8eb21572e122 with sample_3494 has been placed.
order 02645ef2-3890-457a-b878-3499e1e1b30f with sample_3499 has been placed.
order 783d8b68-71a2-4424-91c0-09031c199036 with sample_3501 has been placed.
order b0a8243f-288b-4dbb-95b7-a2d9a10d3feb with sample_3502 has been placed.
order 3e380c18-fa85-45f3-bd2b-f1def8cad0e2 with sample_3496 has been placed.
order 74e3bb01-e919-4081-8517-106b984e2d13 with sample_3504 has been placed.
order f7e213d9-ff75-4d07-991d-b0670a82ed06 with sample_3503 has been placed.
order da8d429c-4f9a-44c8-8bd0-3bd5b1bd65d8 with sample_3500 has been placed.
order 6d752499-c3c8-4fcd-8166-76abdf1d3f43 with sample_3507 has been placed.
order 3e8edede-7301-4ee2-88ca-9d7f68be4c8e with sample_3511 has been placed.
order 877aa9da-e3ea-42f8-b397-e99ba7d12a3e with sample_3510 has been placed.
order 2b907a05-c3b5-40e9-99e5-5dd061571168 with sample_3512 has been placed.

order 07bb771c-1b55-410f-92f7-58f318aad76b with sample_3604 has been placed.
order f893b636-e362-4452-b281-ed7f0b0d4d95 with sample_3599 has been placed.
order 17c85a17-3923-4c52-b159-f692f579512e with sample_3605 has been placed.
order 1c3da164-e999-4d8d-b1a6-4f5b0faa251e with sample_3603 has been placed.
order 1ef08bad-a106-4143-8dcd-8514a614bee6 with sample_3608 has been placed.
order 4d183485-eda6-4864-99ae-8950c62e95fd with sample_3610 has been placed.
order 62876b78-6bf6-44b3-b27e-b68ab9ee31a6 with sample_3609 has been placed.
order c00bc8b7-4523-489d-acb1-a296598d2624 with sample_3607 has been placed.
order 369210f1-ad27-4cc6-ab09-b703febfbb79 with sample_3601 has been placed.
order c9e005af-c2dd-4ec1-996d-228c4bab47a1 with sample_3618 has been placed.
order 998180f0-6b12-40d7-b8da-588243b7a1aa with sample_3616 has been placed.
order 3fc92312-5249-489a-b3e0-bc8b7c107391 with sample_3617 has been placed.
order d634628b-88c1-4d3a-87f4-54b4c8efd118 with sample_3613 has been placed.

order 221f8c93-8779-47f4-bba8-f4d25163ec60 with sample_3711 has been placed.order 2f13d24e-e886-4f2f-9ea6-301683e53219 with sample_3710 has been placed.

order 5ead684a-2499-4b74-97f7-2199c510fc85 with sample_3714 has been placed.
order d33321a6-526d-4ddb-b925-89f2315fc225 with sample_3712 has been placed.
order b56c2a17-7814-4b3b-8b41-67d1843cbd86 with sample_3713 has been placed.
order cda31565-c6f9-434c-a834-c313597b0a8d with sample_3701 has been placed.
order f94ed901-6231-4d14-93e7-52112fc5dc0e with sample_3716 has been placed.
order a04e983b-2132-41b4-a259-9a86d84c79aa with sample_3717 has been placed.
order 3e1fea20-81a0-48eb-a0a1-8b90f4dabd7a with sample_3715 has been placed.
order 21fa29d0-0a80-480d-a193-b3dbe78a8f14 with sample_3718 has been placed.
order 267aea79-9bd9-42c9-8f6d-6bbeb329dc39 with sample_3719 has been placed.
order d3975d1e-c902-4a66-9eba-83ab6c67c538 with sample_3725 has been placed.
order 8f0e3273-d31b-4fb1-bef6-70bc1063746a with sample_3724 has been placed.

order ecb9b2bd-0e1a-435e-98cc-98d55c9dba29 with sample_3809 has been placed.
order 2e1e48b1-c0c5-4911-977f-55924133f7ca with sample_3818 has been placed.
order 6d9577e5-9bda-4144-9bfb-5c6b91b866b1 with sample_3820 has been placed.
order 40d2a252-bdb5-47a5-826f-b924a6a49551 with sample_3821 has been placed.
order 0354be77-47e5-4747-8410-eb0b58e5dc70 with sample_3819 has been placed.
order d62d554e-4da6-4c37-95dc-d73a2aedbffa with sample_3823 has been placed.
order c8b6cc58-53ed-4230-b2d4-f0a680c5deb1 with sample_3822 has been placed.
order 2cba07ca-9e69-4385-b14f-eaa00579b6c3 with sample_3814 has been placed.
order a2c53463-c0bf-40e9-b777-93ce3ea9ec55 with sample_3816 has been placed.
order b7072cee-8593-481c-985c-e7a1d90babb1 with sample_3817 has been placed.
order 6bb722d5-2deb-4c55-bb9e-64e3107ff017 with sample_3825 has been placed.
order 0c96c488-9621-4827-868f-1177cc783ac3 with sample_3815 has been placed.
order 1e0e6933-9f8a-44de-be4a-6db882a1b292 with sample_3824 has been placed.

order e06de3cf-fc6e-4815-aec3-13b33f755105 with sample_3924 has been placed.
order f30e06b4-59c1-421c-9d1c-03bdd5dfe3a6 with sample_3919 has been placed.
order 2cd52958-75ba-41c3-9cda-efa5447aaf48 with sample_3926 has been placed.
order 123c7476-d24a-4019-b36d-d2ff2e3b4385 with sample_3927 has been placed.
order b170b882-ab8d-497c-a14d-089c16993bb3 with sample_3925 has been placed.
order ebc3a579-ceb3-4a47-8d9f-5325602ebfad with sample_3922 has been placed.
order cbdc1f37-29d6-41d4-b092-1bdc95bc0028 with sample_3928 has been placed.
order 9ed1f97c-31c7-4e93-8574-b4fbfc748e03 with sample_3921 has been placed.
order b853315d-abb7-4090-83e5-845523d6254d with sample_3916 has been placed.
order 90cf9b77-ba2a-4606-be75-b2385e1ff0d1 with sample_3923 has been placed.
Finished
CPU times: user 6.56 s, sys: 4.01 s, total: 10.6 s
Wall time: 36min 12s


### Request order
<font color='red'>The following lines will start the order in the planet server, once the order is placed and running, there is no way to stop it.</font>

NOTE: The following loop will skip the samples that have already been downloaded, however it's based on the existing_orders request, and we are not sure how long the requests will remain in the planet server.

In [69]:
# Request the existing orders and store their sample_id (name)
current_server_orders = get_existing_orders(client)
ordered_sample_names = [order['name'] for order in current_server_orders]

now = datetime.datetime.now()
formated_now = now.strftime('%Y%m%d_%H_%M')
ordr_log_file = os.path.join(LOG_PATH, f'order_logs_{formated_now}.txt')


orders_info = []
pbar = tqdm(total=len(orders))
for new_order in orders:

    # Make sure that the sample is not already downloaded
    sample_name = new_order['name']
    if sample_name not in ordered_sample_names:
        
        try:
            # The following line will create the order in the server
            @backoff.on_exception(backoff.expo,planet.api.exceptions.OverQuota, max_time=360)
            def place_order():
                return client.create_order(new_order).get()
            
            order_info = place_order()
            orders_info.append(order_info)
            
            order_id = order_info['id']
            sample_name = order_info['name']
            
            print(f'order {order_id} with {sample_name} has been placed.')
            
        except Exception as e:
            with open(ordr_log_file, 'a') as lf:
                print(f'there was an error with the sample {sample_name}, please check the log files.')
                lf.write(f'Sample {sample_name}:{e}\n')
    pbar.update(1)
pbar.close()
print('Finished')

order 3b4232de-6adf-4d96-9423-1880f17fcf1b with sample_2 has been placed.
order fe4dc84e-d175-4f3b-8eb4-788ffae1674f with sample_3 has been placed.
order 96663a20-fc06-4e13-99c8-ec92acaaa457 with sample_4 has been placed.
order 24e15050-3b0e-4839-b072-b49d0d10ece7 with sample_5 has been placed.
order 3ab76070-0f09-4618-9a03-4aa88d183612 with sample_6 has been placed.
order 9c3e6b32-6d5b-417b-9fd0-cd8fcfd6c713 with sample_7 has been placed.
order ea10dc15-3b1a-4972-a9a8-551f3ee3fae8 with sample_101 has been placed.
order 9dfcb6c8-5ff7-454d-8f03-afce32b9c14a with sample_102 has been placed.
order 5b5e1e2c-023d-4038-ba24-1beb3787a49f with sample_103 has been placed.
order 96e9095e-1a6a-4bee-8615-4fd71bcb1874 with sample_1975 has been placed.
there was an error with the sample sample_3201, please check the log files.

Finished


## Get status
The following get_order_status line has to be re-runned everytime we want to know the orders statusw

In [58]:
# Use the pages to limit the search, every page will display 20 orders.
current_orders = get_orders_status(client, pages=None).sort_values(by=['created_on'])

In [70]:
current_orders

,created_on,last_message,last_modified,id,name,state


# 3. Download

In [ ]:
%%time
current_server_orders = get_existing_orders(client, pages=None)
len(current_server_orders)

In [ ]:
# Define the dates in which the desired oreders were ordered.

now = datetime.datetime.now()
formated_now = now.strftime('%Y%m%d_%H_%M')
dw_log_file = os.path.join(LOG_PATH, f'download_logs_{formated_now}.txt')

# Select the dates in which you want to download the images
start_date = datetime.date(2020,9,1)
stop_date = datetime.date(2020,9,20)
success_states = ['success', 'partial']

for order in current_server_orders:
    
    created_on = pd.to_datetime(order['created_on']).date()
    state = order['state']

    if state in success_states:

        if created_on >= start_date and created_on <= stop_date:
            # Create the download folder
            download_order_path = os.path.join(DOWNLOAD_PATH, 'budi_wardhana', order['name'])
            i+=1
            Path(download_order_path).mkdir(parents=True, exist_ok=True)

            # Check if there is a .zip file inside the folder
            if not any(['.zip' in f for f in os.listdir(download_order_path)]):
                try:
                    sample_name = order["name"]
                    print(f'downloading {sample_name} ')
                    callback = api.write_to_file(directory=f'{download_order_path}/', overwrite=True)

                    @backoff.on_exception(backoff.expo,planet.api.exceptions.OverQuota,max_time=360)
                    def download():
                        return client.download_order(order['id'], callback=callback)
                    responses = download()
                    sleep(0.5)
                    
                except Exception as e:
                    print(f'There was an error with {sample_name}, please check download log file.')
                    with open(dw_log_file, 'w') as lf:
                        lf.write(f'Sample {sample_name}:{e}\n')
print('done!')

# 4. Additional commands

In [63]:

callback = api.write_to_file(directory=os.path.join(os.getcwd(), 'downloads'), overwrite=True)
client.download_order('ceca46ca-f55f-4b6b-8e5c-fde5ae57f6d3', callback=callback)

In [ ]:
failed_samples = [x[0] for x in get_no_access_assets_from_log('logs/order_logs_20200915_12_41_yelena.txt')]
len(failed_samples)

In [ ]:
failed_items_ids = [x[1] for x in get_no_access_assets_from_log('logs/order_logs_20200915_12_41_yelena.txt')]
failed_items_ids